In [144]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split

#Importando o modelo machine learnig
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

In [145]:
# importar csv
data = pd.read_csv("nge_dados_input.csv")


In [146]:
data.columns

Index(['id', 'entidade', 'fornecedor', 'complexidade', 'stat_pagto',
       'qtd_pag_word', 'qtd_carc', 'qtd_tabela', 'qtd_image', 'pag_final'],
      dtype='object')

In [147]:
# prepara dados de entrada X e dados de saida Y
colunas_X= ['entidade', 'fornecedor', 'complexidade', 'stat_pagto',
       'qtd_pag_word', 'qtd_carc', 'qtd_tabela',
       'qtd_image']
colunas_y=['pag_final']
#colunas_y=['id',  'pag_final']

X = data[colunas_X]
y = data[colunas_y]

In [148]:
X.tail()

,entidade,fornecedor,complexidade,stat_pagto,qtd_pag_word,qtd_carc,qtd_tabela,qtd_image
28,0,0,1,0,40,29815,349,305
29,0,0,0,0,77,33684,383,348
30,0,0,1,0,24,32209,7,8
31,0,0,0,0,59,125372,363,323
32,0,0,0,0,48,83759,115,205


In [149]:
y.head()

,pag_final
0,1
1,52
2,66
3,86
4,68


In [150]:
#########################################################
# INÍCIO DA PARTE DE MACHINE LEARNIG  \
# RANDOM FOREST
########################################################

In [151]:
# instalar biblioteca scikit-learn
#! pip install -U scikit-learn

In [152]:
# método train_test_split, Scikit-Learn 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [153]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((26, 8), (7, 8), (26, 1), (7, 1))

In [154]:
y_test.head()

,pag_final
31,78
15,88
26,84
17,296
8,30


In [155]:
modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
modelo.fit(X_train, y_train)

<ipython-input-155-957cc5138365>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo.fit(X_train, y_train)


RandomForestClassifier(n_jobs=-1, random_state=0)

In [156]:
p = modelo.predict(X_test)

In [157]:
p

array([ 66,  82,  66, 176,  30,  30,  66])

In [158]:
print(p.shape,y_test.shape)

(7,) (7, 1)


In [159]:
y_test

,pag_final
31,78
15,88
26,84
17,296
8,30
9,22
19,86


In [ ]:
# https://github.com/WalnerPessoa/Medical-Cost-Personal/blob/master/Aula%205%20-%20Regress%C3%A3o%20linear%20para%20prever%20as%20despesas%20m%C3%A9dicas%20de%20um%20paciente%20.ipynb


# Criando uma lista com os nomes dos atributos
#feature_cols = ['age','sex','bmi','children','smoker']

# Usando a lista para escolher um subconjunto do dataframe original
#X = data[feature_cols]

# Selecionando a Série no dataframe
#y = data.charges

# Dividindo entre conjunto de treinamento e teste
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Ajsutando o modelo (Aprendendo os coeficientes)
#linreg.fit(X_train, y_train)

# Fazendo predições no conjunto de teste
#y_pred = linreg.predict(X_test)

# Calculando RMSE para nossas predições
#print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
#########################################################
# INÍCIO DA PARTE DE MACHINE LEARNIG  \
# LINEAR REGRESSION
########################################################

In [160]:
# prepara dados de entrada X e dados de saida Y
colunas_X= [ 'entidade', 'fornecedor', 'complexidade', 'stat_pagto',
       'qtd_pag_word',  'qtd_carc', 'qtd_tabela',
       'qtd_image']
colunas_y=['pag_final']
X = data[colunas_X]
y = data[colunas_y]

In [161]:
# método train_test_split, Scikit-Learn 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [162]:
##### machine learnig

#Instanciando o estimador
linreg = LinearRegression()

#Ajustando o modelo ao conjunto de treino
linreg.fit(X_train, y_train)
print(linreg.intercept_)
print(linreg.coef_)


[6.568489]
[[ 2.67648723e+00 -1.03107131e-11 -9.78503704e-01  2.88657986e-15
   1.17081366e+00  5.76641422e-05  2.96385768e-03 -3.41835709e-03]]


In [163]:
colunas_X

['entidade',
 'fornecedor',
 'complexidade',
 'stat_pagto',
 'qtd_pag_word',
 'qtd_carc',
 'qtd_tabela',
 'qtd_image']

In [164]:
X_train.columns

Index(['entidade', 'fornecedor', 'complexidade', 'stat_pagto', 'qtd_pag_word',
       'qtd_carc', 'qtd_tabela', 'qtd_image'],
      dtype='object')

In [165]:
#Atributos e seus coeficientes
list(zip(colunas_X, linreg.coef_))

[('entidade',
  array([ 2.67648723e+00, -1.03107131e-11, -9.78503704e-01,  2.88657986e-15,
          1.17081366e+00,  5.76641422e-05,  2.96385768e-03, -3.41835709e-03]))]

In [166]:
#Fazendo predições no conjunto de teste
y_pred = linreg.predict(X_test)

In [167]:
y_pred

array([[ 28.80676276],
       [ 88.90598845],
       [ 99.22256115],
       [ 62.13157205],
       [ 82.84771447],
       [ 69.97274573],
       [ 60.75170346],
       [120.17350842],
       [113.05549791]])

In [143]:
y_test

,pag_final
14,38
19,86
3,86
27,54
32,66
26,84
20,68
25,84
24,82
